In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import MetaData

import psycopg2
import psycopg2.extras
import json
import os

import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement

credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        try:
            db = create_engine('postgresql+psycopg2://'+db_user+':'+db_pw+'@'+host+'/'+default_db, echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

In [66]:
db, conn = pgconnect(credentials)

Connected successfully.


In [30]:
def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(sqlcmd, args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

In [31]:
def create_wkt_element(geom, srid):
    if geom.geom_type == 'Polygon':
        geom = MultiPolygon([geom])
    return WKTElement(geom.wkt, srid)

In [32]:
### SA2 Data

SA2 = gpd.read_file("SA2_2016_AUST/SA2_2016_AUST.shp")

SA2.head

# Make all column names lowercase
SA2.columns = SA2.columns.str.lower()

# Filter rows that are only part of Greater Sydney
SA2 = SA2[SA2["gcc_name16"].str.contains("Greater Sydney") == True]

# Removing unnecessary columns
SA2 = SA2.drop(columns=['gcc_code16', 'gcc_name16', 'ste_code16', 'ste_name16'])

# Checked for entries with no geometry, which are then removed
null_geom = []

for index, row in SA2.iterrows():
    if row['geometry'] is None:
        null_geom.append(index)

SA2.drop(null_geom, axis=0, inplace=True)

# Convert the values from object to more usable forms
SA2['sa2_main16'] = SA2['sa2_main16'].astype('int64')
SA2['sa2_5dig16'] = SA2['sa2_5dig16'].astype('int64')
SA2['sa2_name16'] = SA2['sa2_name16'].astype('string')
SA2['sa3_code16'] = SA2['sa3_code16'].astype('int64')
SA2['sa3_name16'] = SA2['sa3_name16'].astype('string')
SA2['sa4_code16'] = SA2['sa4_code16'].astype('int64')
SA2['sa4_name16'] = SA2['sa4_name16'].astype('string')

# make the column names neater - remove the 16 from the column headings, as we kow all the data is from 2016
SA2.rename(
    columns={'sa2_main16': 'sa2_code', 'sa2_5dig16': 'sa2_5digit', 'sa2_name16': 'sa2_name', 'sa3_code16': 'sa3_code',
             'sa3_name16': 'sa3_name', 'sa4_code16': 'sa4_code', 'sa4_name16': 'sa4_name', 'areasqkm16': 'areasqkm', 'gcc_code16': 'gcc_code', 'gcc_name16':'gcc_name', 'ste_code16':'ste_code', 'ste_name16':'ste_name'},
    inplace=True)


srid = 4283  # Change ID to Australian coordinate system

# convert for postGIS
SA2['geom'] = SA2['geometry'].apply(lambda x: create_wkt_element(x, srid))  # converting to WKT format
SA2 = SA2.drop(columns="geometry")  # deleting the old copy
SA2

,sa2_code,sa2_5digit,sa2_name,sa3_code,sa3_name,sa4_code,sa4_name,areasqkm,geom
27,102011028,11028,Avoca Beach - Copacabana,10201,Gosford,102,Central Coast,6.4376,MULTIPOLYGON (((151.4137275700001 -33.46559324...
28,102011029,11029,Box Head - MacMasters Beach,10201,Gosford,102,Central Coast,32.0860,MULTIPOLYGON (((151.3539836770001 -33.49854010...
29,102011030,11030,Calga - Kulnura,10201,Gosford,102,Central Coast,767.9512,MULTIPOLYGON (((151.204595054 -33.532982995999...
30,102011031,11031,Erina - Green Point,10201,Gosford,102,Central Coast,33.7934,MULTIPOLYGON (((151.3679483220001 -33.43822416...
31,102011032,11032,Gosford - Springfield,10201,Gosford,102,Central Coast,16.9124,MULTIPOLYGON (((151.310060818 -33.426989095999...
...,...,...,...,...,...,...,...,...,...
571,128021537,11537,Royal National Park,12802,Sutherland - Menai - Heathcote,128,Sydney - Sutherland,139.3336,MULTIPOLYGON (((151.071650994 -34.057945007999...
572,128021538,11538,Sutherland - Kirrawee,12802,Sutherland - Menai - Heathcote,128,Sydney - Sutherland,7.7547,MULTIPOLYGON (((151.05049439 -34.0200584319999...
573,128021607,11607,Engadine,12802,Sutherland - Menai - Heathcote,128,Sydney - Sutherland,8.9538,MULTIPOLYGON (((150.9956780320001 -34.05362100...
574,128021608,11608,Loftus - Yarrawarrah,12802,Sutherland - Menai - Heathcote,128,Sydney - Sutherland,3.8436,MULTIPOLYGON (((151.0395427830001 -34.04175791...


In [39]:
### Neighbourhoods data

neighbourhoods = pd.read_csv("Neighbourhoods.csv")


# set empty cells to NaN using numpy
neighbourhoods = neighbourhoods.replace('', np.nan, regex=True)

# remove regions not an sa2
not_sa2_region = []
for index, row in neighbourhoods.iterrows():
    if row['area_id'] not in list(SA2['sa2_code']):
        not_sa2_region.append(index)

neighbourhoods.drop(not_sa2_region, axis=0, inplace=True)

neighbourhoods.rename(columns={'area_id': 'sa2_code', 'area_name': 'sa2_name'}, inplace=True)

neighbourhoods

,Unnamed: 0,sa2_code,sa2_name,land_area,population,number_of_dwellings,number_of_businesses,median_annual_household_income,avg_monthly_rent,0-4,5-9,10-14,15-19
0,0,102011028,Avoca Beach - Copacabana,643.8,7590,2325,738.0,46996.0,1906.0,467,583,604,560
1,1,102011029,Box Head - MacMasters Beach,3208.6,10986,3847,907.0,42621.0,1682.0,586,696,661,692
2,2,102011030,Calga - Kulnura,76795.1,4841,1575,1102.0,42105.0,1182.0,220,254,304,320
3,3,102011031,Erina - Green Point,3379.3,14237,4450,1666.0,43481.0,1595.0,695,778,916,838
4,4,102011032,Gosford - Springfield,1691.2,19385,6373,2126.0,45972.0,1382.0,1200,1079,963,977
...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,307,128021537,Royal National Park,13933.4,28,4,3.0,NaN,1899.0,0,4,0,4
308,308,128021538,Sutherland - Kirrawee,775.5,21760,6924,2086.0,53378.0,1809.0,1538,1242,1081,1033
309,309,128021607,Engadine,895.4,17424,5219,1121.0,52667.0,1955.0,1333,1529,1283,1031
310,310,128021608,Loftus - Yarrawarrah,384.4,7426,2177,417.0,51243.0,2163.0,503,572,532,447


In [40]:
### Business data

business_stats = pd.read_csv("BusinessStats.csv")

# remove regions not in sa2
not_sa2_region = []
for index, row in business_stats.iterrows():
    if row['area_id'] not in list(SA2['sa2_code']):
        not_sa2_region.append(index)

business_stats.drop(not_sa2_region, axis=0, inplace=True)

# convert to string
business_stats['area_name'] = business_stats['area_name'].astype('string')

business_stats.rename(
    columns={'area_id': 'sa2_code', 'area_name': 'sa2_name'}, inplace=True)

business_stats

,sa2_code,sa2_name,number_of_businesses,accommodation_and_food_services,retail_trade,agriculture_forestry_and_fishing,health_care_and_social_assistance,public_administration_and_safety,transport_postal_and_warehousing
27,102011028,Avoca Beach - Copacabana,724,33,35,4,60,0,19
28,102011029,Box Head - MacMasters Beach,934,23,45,13,43,3,28
29,102011030,Calga - Kulnura,1134,14,43,231,12,0,55
30,102011031,Erina - Green Point,1712,68,145,19,179,12,43
31,102011032,Gosford - Springfield,2233,94,167,9,246,10,97
...,...,...,...,...,...,...,...,...,...
571,128021537,Royal National Park,3,3,0,0,0,0,0
572,128021538,Sutherland - Kirrawee,2136,73,113,6,127,17,122
573,128021607,Engadine,1106,37,69,3,82,9,51
574,128021608,Loftus - Yarrawarrah,452,9,20,3,25,5,31


In [41]:
### Break and Enter data

break_and_enter = gpd.read_file("break_and_enter/BreakEnterDwelling_JanToDec2021.shp")

# make all columns lower case
break_and_enter.columns = break_and_enter.columns.str.lower()

# cast density as string
break_and_enter['density'] = break_and_enter['density'].astype('string')

# change geometry datatype as with SA2 data
srid = 4283  # this is the id of the Australian coordinate system
break_and_enter['geom'] = break_and_enter['geometry'].apply(lambda x: create_wkt_element(x, srid))  # applying the function
break_and_enter = break_and_enter.drop(columns=['contour', 'orig_fid', "geometry"]) # removing columns that are not needed

# rename columns to correct formatting
break_and_enter.rename(
    columns={'shape_leng': 'shape_length', 'objectid': 'object_id'}, inplace=True)

break_and_enter

,object_id,density,shape_length,shape_area,geom
0,1,Low Density,0.012138,0.000006,MULTIPOLYGON (((149.9107801470001 -37.06636313...
1,2,Low Density,0.019106,0.000015,MULTIPOLYGON (((149.9060088220001 -37.05836717...
2,3,Low Density,0.006068,0.000002,MULTIPOLYGON (((148.9424993900001 -37.04208530...
3,4,Low Density,0.011006,0.000006,MULTIPOLYGON (((149.1958326340001 -36.91833255...
4,5,Low Density,0.008073,0.000003,MULTIPOLYGON (((149.8483600650001 -36.68458912...
...,...,...,...,...,...
2589,2590,High Density,0.015358,0.000011,MULTIPOLYGON (((153.2796105350001 -28.55387845...
2590,2591,High Density,0.028327,0.000029,MULTIPOLYGON (((150.3593920080001 -28.60530123...
2591,2592,High Density,0.006654,0.000002,MULTIPOLYGON (((153.568248161 -28.383068159999...
2592,2593,High Density,0.047098,0.000044,MULTIPOLYGON (((153.5391676050001 -28.18857974...


In [63]:
### Catchments data

# reading in all files
catchments_future = gpd.read_file("catchments/catchments_future.shp")
catchments_primary = gpd.read_file("catchments/catchments_primary.shp")
catchments_secondary = gpd.read_file("catchments/catchments_secondary.shp")

# combine the 3 catchment shapefiles into one and remove unnecessary columns
all_catchments = pd.concat([catchments_primary, catchments_secondary])
all_catchments.drop(columns=['PRIORITY'], inplace=True)
all_catchments = pd.concat([all_catchments, catchments_future])
all_catchments.drop(columns=['ADD_DATE'], inplace=True)

# make all rows and columns lower case 
all_catchments.columns = all_catchments.columns.str.lower()

all_catchments['use_id'] = all_catchments['use_id'].astype('int64')
all_catchments['catch_type'] = all_catchments['catch_type'].astype('string')
all_catchments['use_desc'] = all_catchments['use_desc'].astype('string')

all_catchments[
    ['kindergart', 'year1', 'year2', 'year3', 'year4', 'year5', 'year6', 'year7', 'year8', 'year9', 'year10', 'year11',
     'year12']] = all_catchments[
    ['kindergart', 'year1', 'year2', 'year3', 'year4', 'year5', 'year6', 'year7', 'year8', 'year9', 'year10', 'year11',
     'year12']].eq('yes').mul(1)

# cast again
all_catchments['kindergart'] = all_catchments['kindergart'].astype('int64')
all_catchments['year1'] = all_catchments['year1'].astype('int64')
all_catchments['year2'] = all_catchments['year2'].astype('int64')
all_catchments['year3'] = all_catchments['year3'].astype('int64')
all_catchments['year4'] = all_catchments['year4'].astype('int64')
all_catchments['year5'] = all_catchments['year5'].astype('int64')
all_catchments['year6'] = all_catchments['year6'].astype('int64')
all_catchments['year7'] = all_catchments['year6'].astype('int64')
all_catchments['year8'] = all_catchments['year6'].astype('int64')
all_catchments['year9'] = all_catchments['year6'].astype('int64')
all_catchments['year10'] = all_catchments['year6'].astype('int64')
all_catchments['year11'] = all_catchments['year6'].astype('int64')
all_catchments['year12'] = all_catchments['year6'].astype('int64')


# make polygons nice
all_catchments['geom'] = all_catchments['geometry'].apply(lambda x: create_wkt_element(x, srid))  # applying the function
all_catchments = all_catchments.drop(columns="geometry")  # deleting the old copy

#
all_catchments.rename(
    columns={'kindergart': 'kindergarten'}, inplace=True)

#
aggregation_functions = {'use_id': 'first', 'kindergarten': 'sum', 'year1': 'sum', 'year2': 'sum', 'year3': 'sum',
                         'year4': 'sum',
                         'year5': 'sum', 'year6': 'sum', 'year7': 'sum', 'year8': 'sum', 'year9': 'sum',
                         'year10': 'sum', 'year11': 'sum', 'year12': 'sum', 'catch_type': 'first',
                         'use_desc': 'first', 'geom': 'first'}
all_catchments = all_catchments.groupby(all_catchments['use_id']).aggregate(aggregation_functions)

all_catchments

,use_id,kindergarten,year1,year2,year3,year4,year5,year6,year7,year8,year9,year10,year11,year12,catch_type,use_desc,geom
use_id,,,,,,,,,,,,,,,,,
1001,1001,0,0,0,0,0,0,0,0,0,0,0,0,0,PRIMARY,Abbotsford PS,MULTIPOLYGON (((151.1302395408945 -33.84390202...
1002,1002,0,0,0,0,0,0,0,0,0,0,0,0,0,PRIMARY,Aberdeen PS,MULTIPOLYGON (((151.0719795800697 -32.02238135...
1003,1003,0,0,0,0,0,0,0,0,0,0,0,0,0,PRIMARY,Abermain PS,MULTIPOLYGON (((151.4285214252146 -32.74415247...
1007,1007,0,0,0,0,0,0,0,0,0,0,0,0,0,PRIMARY,Adaminaby PS,MULTIPOLYGON (((148.8921134227744 -35.80242311...
1008,1008,0,0,0,0,0,0,0,0,0,0,0,0,0,PRIMARY,Adamstown PS,MULTIPOLYGON (((151.7350932675866 -32.93599678...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8915,8915,0,0,0,0,0,0,0,0,0,0,0,0,0,HIGH_COED,Armidale SC,MULTIPOLYGON (((152.3027050368868 -30.29705826...
8916,8916,0,0,0,0,0,0,0,0,0,0,0,0,0,HIGH_COED,Oran Park HS,MULTIPOLYGON (((150.7545229377899 -33.95471945...
8917,8917,0,0,0,0,0,0,0,0,0,0,0,0,0,HIGH_COED,Murrumbidgee RHS,MULTIPOLYGON (((146.0364140876778 -33.70653837...


In [50]:
### Walking count sites data

walking_count_sites = gpd.read_file("Walking_count_sites.geojson")

walking_count_sites.columns = walking_count_sites.columns.str.lower()

walking_count_sites['location'] = walking_count_sites['location'].astype('string')
walking_count_sites['sitedescription'] = walking_count_sites['sitedescription'].astype('string')

walking_count_sites['geom'] = walking_count_sites['geometry'].apply(lambda x: WKTElement(x.wkt, srid=srid))
walking_count_sites.rename(
    columns={'objectid': 'object_id', 'sitedescription': 'site_description', 'geometry': 'geom'}, inplace=True)


walking_count_sites

,object_id,site_id,location,site_description,geom,geom
0,1,2,Botany Road,Between Bourke Street and Hansard Street,POINT (151.20287 -33.90732),POINT (151.20286600009 -33.9073219995779)
1,2,3,King Street,Between Whitehorse Street and Newman Street,POINT (151.17795 -33.89934),POINT (151.177947185903 -33.8993387264436)
2,3,4,William Street,Between Crown Street and Palmer Street,POINT (151.21666 -33.87438),POINT (151.2166614819 -33.874375432462)
3,4,5,City Road,Between Carillon Avenue and Forbes Street,POINT (151.18803 -33.89170),POINT (151.18802777906 -33.8916968892733)
4,5,6,Broadway,Between Buckland Street and Abercrombie Street,POINT (151.19771 -33.88434),POINT (151.197710966834 -33.8843416781514)
...,...,...,...,...,...,...
115,116,118,Bridge Street,Between Loftus Street and Pitt Street,POINT (151.20968 -33.86356),POINT (151.209679006248 -33.8635647044744)
116,117,119,Margaret Street,Between George Street and Carrington Street,POINT (151.20689 -33.86512),POINT (151.206891862077 -33.8651194710191)
117,118,120,Margaret Street,Between Clarence Street and Kent Street,POINT (151.20452 -33.86516),POINT (151.204522223741 -33.8651608667998)
118,119,121,York Street,Between Barrack Street and King Street,POINT (151.20595 -33.86805),POINT (151.2059536541 -33.868052889575)


In [52]:
conn.execute("""
DROP TABLE IF EXISTS sa2;
CREATE TABLE sa2(
   sa2_code INTEGER PRIMARY KEY,
   sa2_5digit INTEGER,
   sa2_name VARCHAR(100) UNIQUE,
   sa3_code INTEGER,
   sa3_name VARCHAR(100),
   sa4_code INTEGER,
   sa4_name VARCHAR(100),
   areasqkm FLOAT,
   geom GEOMETRY(MULTIPOLYGON,4283)
);""")


SA2.to_sql('sa2', conn, if_exists='append', index=False,
           dtype={'geom': Geometry(geometry_type='MULTIPLOYGON', srid=4283)})

In [53]:
conn.execute("""
DROP TABLE IF EXISTS neighbourhoods;
CREATE TABLE neighborhoods(
   sa2_code INTEGER PRIMARY KEY REFERENCES sa2(sa2_code),
   sa2_name VARCHAR(100) REFERENCES sa2(sa2_name),
   land_area FLOAT,
   population FLOAT,
   number_of_dwellings FLOAT,
   median_annual_household_income FLOAT,
   avg_monthly_rent FLOAT,
   total_young_people INTEGER
);""")

neighbourhoods.to_sql('neighbourhoods', conn, if_exists='append', index=False)

In [6]:
conn.execute("""
DROP TABLE IF EXISTS business_stats;
CREATE TABLE business_stats(
   sa2_code INTEGER PRIMARY KEY REFERENCES sa2(sa2_code),
   sa2_name VARCHAR(100) REFERENCES sa2(sa2_name),
   number_of_businesses INTEGER,
   accommodation_and_food_services INTEGER,
   retail_trade INTEGER,
   health_care_and_social_assistance INTEGER
);""")

business_stats.to_sql('business_stats', conn, if_exists='append', index=False)

In [56]:
conn.execute("""
DROP TABLE IF EXISTS break_and_enter;
CREATE TABLE break_and_enter(
   object_id INTEGER PRIMARY KEY,
   density varchar(50),
   shape_length FLOAT,
   shape_area FLOAT,
   geom GEOMETRY(MULTIPOLYGON,4283)
);""")

break_and_enter.to_sql('break_and_enter', conn, if_exists='append', index=False,
                       dtype={'geom': Geometry(geometry_type='MULTIPOLYGON', srid=4283)})

In [67]:
conn.execute("""
DROP TABLE IF EXISTS catchments;
CREATE TABLE catchments(
   use_id INTEGER PRIMARY KEY,
   catch_type varchar(50),
   use_desc VARCHAR(50),
   kindergarten INTEGER,
   year1 INTEGER,
   year2 INTEGER,
   year3 INTEGER,
   year4 INTEGER,
   year5 INTEGER,
   year6 INTEGER,
   year7 INTEGER,
   year8 INTEGER,
   year9 INTEGER,
   year10 INTEGER,
   year11 INTEGER,
   year12 INTEGER,
   geom GEOMETRY(MULTIPOLYGON,4283)
);""")

all_catchments.to_sql('catchments', conn, if_exists='append', index=False,
                           dtype={'geom': Geometry(geometry_type='MULTIPOLYGON', srid=4283)})

In [ ]:
def close_connection(conn, db):
    conn.close()
    db.dispose()
    print("Connection closed")